In [1]:
using BenchmarkTools: @btime
import LinearAlgebra as la

In [2]:
include("Basis/basis.jl")
include("Basis/pairing.jl")
include("Basis/HOBasis.jl")
include("Basis/SpinBasis.jl")
include("system.jl")
include("CC/CCSD.jl")
include("CC/CCD.jl")
include("HF/hf.jl")

energy (generic function with 3 methods)

## HO Basis and Spatial System

In [3]:
l = 30
ω = 0.25
basis = SpinBasis(HOBasis(l, ω))

n = 2
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
@time system = System(n, basis, grid, a);

  2.385339 seconds (3.67 M allocations: 410.765 MiB, 9.59% gc time, 72.09% compilation time)


## CCSD

In [ ]:
ccsd = setup_CCSD(system, α = 0.5);

In [ ]:
for i in 1:10
    CCSD_Update!(ccsd);
end

In [ ]:
energy(ccsd)

## CCD

In [ ]:
ccd = setup_CCD(system, α = 0.5);

In [ ]:
@time CCD_Update!(ccd);

In [ ]:
energy(ccd)

# HF

In [5]:
hf = setup_HF(system);

In [12]:
@time HF_update!(hf);

  0.027338 seconds (20 allocations: 121.516 KiB)


In [ ]:
energy(hf)

## Pairing Basis and Pairing System

In [ ]:
l = 8
g = 0.1
basis = Pairing(l, g)

n = 4
system = System(n, basis);

## Testing

In [39]:
spfs = hf.system.spfs;
C = hf.C;
C_tilde = C';
l = hf.system.l
h = hf.system.h
u = hf.system.u;

In [ ]:
function transform_spfs(spfs, C)
    spfs2 = zero.(spfs)
    l = length(spfs)
    
    for i in 1:l
        psi1 = spfs2[i]
        for j in 1:l
            psi1 .+= C[j, i] .* spfs[j]
        end
    end
    return spfs2
end

In [ ]:
@time spfs2 = transform_spfs(spfs, C);

In [ ]:
function transform_onebody(h, C)
    h2 = zero(h)
    l = size(C)[1]
    
    @inbounds for a in 1:l
        for b in 1:l
            for i in 1:l
                for j in 1:l
                    h2[a, b] += C[i, a] * C[j, b] * h[i, j]
                end
            end
        end
    end
    return h2
end

In [ ]:
@time h2 = transform_onebody(h, C);

In [ ]:
@time h3 = C' * h * C;

In [ ]:
h2 ≈ h3

## Twobody

In [ ]:
function transform_twobody_slow(u, C)
    u2 = zero(u)
    l = length(C[:,1])
    
    for a in 1:l, b in 1:l, c in 1:l, d in 1:l
        for i in 1:l, j in 1:l, k in 1:l, m in 1:l
            @inbounds u2[a, b, c, d] += C[i, a] * C[j, b] * C[k, c] * C[m, d] * u[i, j, k, m]
        end
    end
    return u2
end

In [ ]:
@time u_slow = transform_twobody_slow(u, C);

In [ ]:
function transform_twobody(u, C)
    u2 = zero(u)
    u3 = zero(u)
    l = size(C)[1]
    
    @inbounds for a in 1:l, b in 1:l, c in 1:l
        for i in 1:l
            for j in 1:l
                u2[a, b, c, i] += C[j, i] * u[a, b, c, j]
            end
        end
    end
    
    @inbounds for a in 1:l, b in 1:l, d in 1:l
        for i in 1:l
            for j in 1:l
                u3[a, b, i, d] += C[j, i] * u2[a, b, j, d]
            end
        end
    end
    
    @inbounds for a in 1:l, c in 1:l, d in 1:l
        for i in 1:l
            u2[a, i, c, d] = 0
            for j in 1:l
                u2[a, i, c, d] += C[j, i] * u3[a, j, c, d]
            end
        end
    end
    
    @inbounds for b in 1:l, c in 1:l, d in 1:l
        for i in 1:l
            u3[i, b, c, d] = 0
            for j in 1:l
                u3[i, b, c, d] += C[j, i] * u2[j, b, c, d]
            end
        end
    end
    
    return u3
end

In [ ]:
@time u2 = transform_twobody(u, C);

In [ ]:
u2 ≈ u_py

In [14]:
using PyCall

In [15]:
using TensorOperations

In [187]:
function transform_twobody(u, C)
    u3 = zero(u)
    @tensor u3[a, b, c, i] = u[a, b, c, d] * C[d, i]
    @tensor u3[a, b, i, d] = u3[a, b, c, d] * C[c, i]
    @tensor u3[a, i, c, d] = u3[a, b, c, d] * C[b, i]
    @tensor u3[i, b, c, d] := u3[a, b, c, d] * C[a, i];
    return u3
end

transform_twobody (generic function with 1 method)

In [195]:
@time u3 = transform_twobody(u, C);

  0.628787 seconds (1.05 k allocations: 296.721 MiB, 35.50% gc time)


In [194]:
println(fasit ≈ u3)

true


In [193]:
py"""
import numpy as np

def please(u, C, C_tilde):
    _u = np.tensordot(u, C, axes=(3, 0))
    _u = np.tensordot(_u, C, axes=(2, 0)).transpose(0, 1, 3, 2)
    _u = np.tensordot(_u, C_tilde, axes=(1, 1)).transpose(0, 3, 1, 2)
    _u = np.tensordot(C_tilde, _u, axes=(1, 0))
    return _u
"""
@time fasit = py"please"(u, C, C');

  1.349820 seconds (58 allocations: 98.880 MiB, 0.67% gc time)


In [127]:
findmax(fasit), findmin(fasit)

((0.7914455581213373, CartesianIndex(2, 1, 2, 1)), (-0.7914455581213374, CartesianIndex(2, 1, 1, 2)))

((0.022721234479403062, CartesianIndex(16, 57, 32, 53)), (-0.02125490955530783, CartesianIndex(16, 33, 32, 53)))